# 数値解析第５回課題

### 学籍番号：08B22182　氏名：平山聖輝

課題5

$$
\begin{equation}
\left\{
\begin{alignedat}{3}
    x_1 + x_2 - 2x_3 = 2 \\
    3x_1 - 2x_2 + x_3 = 0 \\
    2x_1 + x_2 - 3x_3 = 1
\end{alignedat}
\right.
\end{equation}
$$

の解 $ x = (x_1, x_2, x_3)^T $ を <br>
* 反復法（ヤコビ法）
* ガウス・サイデル法
を用いて，1回目の繰り返しの近似値 $ ( x_{1}^{(1)}, x_{2}^{(1)}, x_{3}^{(1)} )^T = (1, 0, 0)^T $ から2回目の繰り返しの近似値 $ ( x_{1}^{(2)}, x_{2}^{(2)}, x_{3}^{(2)} ) $ を求めよ．

与えられた連立一次方程式を行列の形 $ Ax = b $ に書き換えると

$$
A = \left[
\begin{array}{ccc}
1 & 1 & -2 \\ 3 & -2 & 1 \\ 2 & 1 & -3
\end{array}
\right]
, b = \left[
\begin{array}{c}
2 \\ 0 \\ 1
\end{array}
\right]
$$


### 反復法（ヤコビ法）

ヤコビ法では与えられた $ Ax =b $ の式における行列 $ A $ を下三角行列 $ L $ ，対角行列 $ D $ ，上三角行列 $ U $ に分解して以下の式変形から繰り返し規則を得て解に近づけるという方法である．

$$
Ax = b
$$
$$
(L + D + U)x = b
$$
$$
Dx^{(k+1)} = b - (L + U)x^{(k)}
$$
$$
\rightarrow x^{(k+1)} = D^{-1} ( b - (L + U)x^{(k)} )
$$

よってまず行列 $ A $ を $ L,D,U $ に分解し，与えられた初期値 $ x^{(1)} $ をもとに繰り返し計算を行うことを考える．各行の計算式は以下

$$
x_{i}^{(k+1)} = \Biggr(b_i - \sum_{j=1}^{i-1}{l_{ij}x_j^{(k)}} - \sum_{j=i+1}^{n}{u_{ij}x_{j}^{(k)}}\Biggr)/d_{ii}
$$

In [103]:
import numpy as np

mat_A = np.array([
    [1.0, 1.0, -2.0],
    [3.0, -2.0, 1.0],
    [2.0, 1.0, -3.0]
])

vec_b = np.array([
    [2.0],
    [0.0],
    [1.0]
])
vec_x1 = np.array([
    [1.0],
    [0.0],
    [0.0]
])

In [105]:
def Jacobi (mat_A, vec_b, vec_x_previous, times):

    # L,U,Dを生成，D=A-L-U
    mat_L = np.tril(mat_A, k=-1) #対角以外の下三角行列をとる
    mat_U = np.triu(mat_A, k=1) #対角以外の上三角行列をとる
    mat_D = mat_A - mat_L - mat_U #D=A-L-U
    
    # k=num_timesになるまで繰り返し計算
    for k in range(1, times):
        # 行数と列数の取得
        col, row = mat_A.shape
        # vec_x_newをcol行1列のベクトルに
        vec_x_new = np.array(np.zeros((col, 1)))

        # 計算式通りにfor文回す
        for i in range(col):
            x_temp = vec_b[i, 0]

            for j in range(i):
                x_temp = x_temp - mat_L[i,j]*vec_x_previous[j,0]

            for j in range(i+1, row, 1):
                x_temp = x_temp - mat_U[i,j]*vec_x_previous[j,0]

            vec_x_new[i,0] = x_temp / mat_D[i,i]

        # vec_x_previousを更新
        vec_x_previous = vec_x_new
        
    return vec_x_new

In [107]:
vec_x_jacobi_2 = Jacobi(mat_A, vec_b, vec_x1, 2)
vec_x_jacobi_2

array([[2.        ],
       [1.5       ],
       [0.33333333]])

よってヤコビ法により求まる $ x^{(2)} $ は以下．

$$
x_J^{(2)} = \left[
\begin{array}{c}
2.0 \\ 1.5 \\ 1/3
\end{array}
\right]
$$

### ガウス・サイデル法

ガウス・サイデル法はヤコビ法の式変形を

$$
Ax=b
$$
$$
(L + D + U)x=b
$$
$$
(D + L)x^{(k+1)} = b - U x^{(k)}
$$

として繰り返し計算する手法である．ヤコビ法と似ているが， $ x_i^{(k+1)} (i \geqq 2) $ の値は掃き出し法の前進消去の要領でひとつ前の値から求めることができるため必要メモリが少なくなる．各要素の計算式は以下．

$$
x_{i}^{(k+1)} = \Biggr( b_i - \sum_{j=0}^{i-1}{l_{ij}x_{i}^{(k+1)}} - \sum_{j=i+1}^{n}u_{ij}x_{i}^{(k)} \Biggr) / d_{ii}
$$

In [109]:
def GaussSeidel(mat_A, vec_b, vec_x_previous, times):

    #L,U,Dをヤコビ法同様に用意する．
    mat_L = np.tril(mat_A, k=-1)
    mat_U = np.triu(mat_A, k=1)
    mat_D = mat_A - mat_L - mat_U

    # k=timesになるまで繰り返し計算
    for k in range(1, times):
        # 行数と列数の取得
        col, row = mat_A.shape
        # vec_x_newをcol行1列のベクトルに
        vec_x_new = np.array(np.zeros((col, 1)))

        # 計算式通りにfor文回す，ほとんどヤコビ法と同じ
        for i in range(col):
            x_temp = vec_b[i, 0]

            for j in range(i):
                # ここだけヤコビ法と異なり，vec_x_newで計算できる！
                x_temp = x_temp - mat_L[i,j]*vec_x_new[j,0]

            for j in range(i+1, row, 1):
                x_temp = x_temp - mat_U[i,j]*vec_x_previous[j,0]

            vec_x_new[i,0] = x_temp / mat_D[i,i]

        # vec_x_previousを更新
        vec_x_previous = vec_x_new
        
    return vec_x_new

In [111]:
vec_x_gs_2 = GaussSeidel(mat_A, vec_b, vec_x1, 2)
vec_x_gs_2

array([[2.],
       [3.],
       [2.]])

よってガウス・サイデル法による $ x^{(2)} $ は以下．

$$
x_{GS}^{(2)} = \left[
\begin{array}{c}
2.0 \\ 3.0 \\ 2.0
\end{array}
\right]
$$

### solve関数による両者の比較

では各手法で何回繰り返し計算を行えば真の値に近づいたといえるのだろうか．真の解は

In [140]:
vec_x = np.linalg.solve(mat_A, vec_b)
vec_x

array([[3.5],
       [7.5],
       [4.5]])

より
$$
x = \left[
\begin{array}{c}
3.5 \\ 7.5 \\ 4.5
\end{array}
\right]
$$

どちらの手法でも100回計算を繰り返すと

In [147]:
vec_x_jacobi_100 = Jacobi(mat_A, vec_b, vec_x1, 100)
vec_x_gs_100 = GaussSeidel(mat_A, vec_b, vec_x1, 100)
print("x_jacobi_100 =\n", vec_x_jacobi_100, "\n")
print("x_gs_100=\n", vec_x_gs_100, "\n")

x_jacobi_100 =
 [[3.49999614]
 [7.49998841]
 [4.49999305]] 

x_gs_100=
 [[3.5]
 [7.5]
 [4.5]] 



とヤコビ法の値は真の値とほんの少し（$ 10^{-6} $ ほどの桁）誤差がある．対してガウス・サイデル法の値は表示上の誤差は表れていない．したがってガウス・サイデル法のほうが真の値に近づくのが速い様子が分かる．200回に繰り返し回数を増やすと

In [150]:
vec_x_jacobi_200 = Jacobi(mat_A, vec_b, vec_x1, 200)
vec_x_gs_200 = GaussSeidel(mat_A, vec_b, vec_x1, 200)
print("x_jacobi_200 =\n", vec_x_jacobi_200, "\n")
print("x_gs_200=\n", vec_x_gs_200, "\n")

x_jacobi_200 =
 [[3.5]
 [7.5]
 [4.5]] 

x_gs_200=
 [[3.5]
 [7.5]
 [4.5]] 



と真の値と一致していることが分かる．偏差を逐一計算して許容誤差を下回ったら繰り返しをやめる関数にすれば真の値を無駄なく計算することが可能になることが分かった．